### Import des modules necéssaires au projet

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import math
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.tree import DecisionTreeRegressor
import shap
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings("ignore")

# Description du Dataset

## Colonnes

1. **customer_unique_id** : Un identifiant unique pour chaque client. Ce champ permet d'associer chaque ligne du dataset à un client spécifique et unique dans la base de données.

2. **days_since_last_order** : Le nombre de jours écoulés depuis la dernière commande passée par le client. Cette valeur est calculée en soustrayant la date de la dernière commande enregistrée de la date de chaque commande.

3. **average_days_diff** : La moyenne des différences en jours entre chaque commande et la commande la plus récente. Cette valeur donne une idée du temps moyen entre les commandes pour chaque client.

4. **std_dev_days_diff** : L'écart-type des différences en jours entre chaque commande et la commande la plus récente. Ce calcul est effectué seulement pour les clients ayant passé plus d'une commande. Il mesure la variabilité des intervalles de temps entre les commandes.

5. **mean_order_reviews_score_of_given_client** : La moyenne des scores d'évaluation des commandes pour chaque client. Cette valeur est calculée en excluant les valeurs nulles pour les scores d'évaluation des commandes et donne une indication de la satisfaction globale des clients.

6. **average_order_price_of_client** : Le prix moyen des commandes pour chaque client. Cette valeur est calculée en excluant les valeurs nulles pour les prix des articles de commande et donne une idée de la dépense moyenne par commande pour chaque client.

7. **non_cancelled_orders_count** : Le nombre de commandes non annulées pour chaque client. Cette valeur indique combien de commandes ont été effectivement réalisées et non annulées.

8. **total_order_price_of_client** : La somme totale des prix des commandes pour chaque client. Cette valeur représente le total des dépenses effectuées par chaque client sur la période couverte par le dataset.

Ces colonnes permettent de comprendre divers aspects du comportement d'achat et de la satisfaction des clients, offrant des insights précieux pour les analyses de données et les prises de décision stratégiques.

## Utilisations possibles du Dataset

Ce dataset peut être utilisé pour plusieurs types d'analyses, notamment :

1. **Segmentation RFM (Récence, Fréquence, Montant)** : Segmenter les clients en fonction de leur engagement récent, la fréquence des commandes et le montant dépensé.
2. **Analyse de la satisfaction client** : Corréler les scores de revue avec d'autres activités de commande pour évaluer la satisfaction.
3. **Modélisation prédictive** : Utiliser des caractéristiques telles que la fréquence des commandes et la récence pour prédire les futurs comportements d'achat.
4. **Analyse géographique** : Examiner les tendances des commandes en fonction des régions, en utilisant les codes postaux.

## Limitations

- **Données Manquantes** : Si des données sont manquantes, par exemple les prix ou les scores des revues, cela pourrait biaiser les moyennes et les écarts-types calculés.
- **Outliers** : Les valeurs aberrantes peuvent déformer les moyennes et les écarts-types, nécessitant une analyse plus approfondie ou le nettoyage des données.


#### Requêtes SQL utilisées pour générer le jeu de données

##### Requête 1 : 

##### Requête 2 : 

##### Requête 3 : 

##### Requête 4 : 

##### Requête 5 : 

### Import des données

In [2]:
file_path_1 = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Projet_3/DATA/data_main_1.csv"
file_path_2 = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Projet_3/DATA/data_main_2.csv"
file_path_3 = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Projet_3/DATA/data_main_3.csv"
file_path_4 = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Projet_3/DATA/data_main_4.csv"
file_path_5 = "C:/Users/pdevi/OneDrive/Desktop/OpenClassrooms/Projet_3/DATA/data_main_5.csv"

data_1=pd.read_csv(file_path_1)
print(data_1.shape)
data_2=pd.read_csv(file_path_2)
print(data_2.shape)
data_3=pd.read_csv(file_path_3)
print(data_3.shape)
data_4=pd.read_csv(file_path_4)
print(data_4.shape)
data_5=pd.read_csv(file_path_5)
print(data_5.shape)

(96096, 4)
(96096, 2)
(96096, 2)
(95560, 2)
(95420, 2)


#### Combinaison des données : 

Pour bien combiner les données, on veut s'assurer que les valeurs soient assignées au bon client et ce pour chaque ligne et donc on utilise l'indentifiant unique du client soit 'customer_unique_id' pour reconstruire le jeu de données.

In [3]:
# List of dataframes
dataframes = [data_1, data_2, data_3, data_4, data_5]

# Find the dataframe with the smallest number of rows
smallest_df = min(dataframes, key=len)

# Get common customer_unique_id values based on the smallest dataframe
common_ids = set(smallest_df['customer_unique_id'])

# Filter all dataframes to only include rows with customer_unique_id in common_ids
filtered_dataframes = [df[df['customer_unique_id'].isin(common_ids)] for df in dataframes]

# Merge all dataframes on 'customer_unique_id'
data = filtered_dataframes[0]
for df in filtered_dataframes[1:]:
    data = pd.merge(data, df, on='customer_unique_id', how='inner')

In [4]:
data

customer_unique_id  days_since_last_order  \
0      ffffd2657e2aad2907e67c3e9daecbeb             532.883021   
1      ffff5962728ec6157033ef9805bacc48             168.092095   
2      ffff371b4d645b6ecea244b27531430a             617.070162   
3      fffea47cd6d3cc0a88bd621562a9d061             310.890532   
4      fffcf5a5ff07b0908bd4e2dbc735a684             495.853958   
...                                 ...                    ...   
94984  0004aac84e0df4da2b147fca70cf8255             336.905972   
94985  0000f6ccb0745a6a4b88665a16c9f078             369.875428   
94986  0000f46a3911fa3c0805444483337064             585.850868   
94987  0000b849f77a49e4a4ce2b2a4ca5be3f             163.263090   
94988  0000366f3b9a7992bf8c76cfdf3221e2             160.273507   

       average_days_diff  std_dev_days_diff  \
0             532.883021                0.0   
1             168.092095                0.0   
2             617.070162                0.0   
3             310.890532                0.0   
4             495.853958                0.0   
...                  ...                ...   
94984         336.905972                0.0   
94985         369.875428                0.0   
94986         585.850868                0.0   
94987         163.263090                0.0   
94988         160.273507                0.0   

       mean_order_reviews_score_of_given_client  \
0                                           5.0   
1                                           5.0   
2                                           5.0   
3                                           4.0   
4                                           5.0   
...                                         ...   
94984                                       5.0   
94985                                       4.0   
94986                                       3.0   
94987                                       4.0   
94988                                       5.0   

       average_order_price_of_client  non_cancelled_orders_count  \
0                              56.99                           1   
1                             115.00                           1   
2                              89.90                           1   
3                              64.89                           1   
4                             785.00                           1   
...                              ...                         ...   
94984                         180.00                           1   
94985                          25.99                           1   
94986                          69.00                           1   
94987                          18.90                           1   
94988                         129.90                           1   

       total_order_price_of_client  
0                            56.99  
1                           115.00  
2                            89.90  
3                            64.89  
4                          1570.00  
...                            ...  
94984                       180.00  
94985                        25.99  
94986                        69.00  
94987                        18.90  
94988                       129.90  

[94989 rows x 8 columns]

In [5]:
data.columns

Index(['customer_unique_id', 'days_since_last_order', 'average_days_diff',
       'std_dev_days_diff', 'mean_order_reviews_score_of_given_client',
       'average_order_price_of_client', 'non_cancelled_orders_count',
       'total_order_price_of_client'],
      dtype='object')

Beaucoup de clients n'ont qu'une seule commande, il est donc important de visualiser les nombres de clients dans différentes intervalles de commandes non annulées

In [6]:
# Définir les intervalles de nombre de commandes
bins = [0, 1, 2, 3, 5, 10, 20, 50, 100]
labels = ['0', '1', '2-3', '4-5', '6-10', '11-20', '21-50', '51-100']

# Catégoriser les nombres de commandes dans les intervalles définis
order_intervals = pd.cut(data['non_cancelled_orders_count'], bins=bins, labels=labels, right=False)

# Compter le nombre de clients dans chaque intervalle
customer_counts = order_intervals.value_counts().sort_index()

# Afficher le résultat
print(customer_counts)

0             0
1         92070
2-3        2683
4-5         217
6-10         18
11-20         1
21-50         0
51-100        0
Name: non_cancelled_orders_count, dtype: int64


Une grande majorité des clients n'a qu'une seule commande, et environ 3% des clients ont plus d'une seule commande. Cette petite valeur ne sera pas négligeable lorsque nous devrons segmenter les clients en différents groupes.

In [7]:
# Calculer la proportion de valeurs manquantes pour chaque colonne
missing_values_proportion = data.isna().mean() * 100

# Afficher la proportion de valeurs manquantes par colonne
print("Proportion de valeurs manquantes par colonne (%):")
print(missing_values_proportion)

Proportion de valeurs manquantes par colonne (%):
customer_unique_id                          0.0
days_since_last_order                       0.0
average_days_diff                           0.0
std_dev_days_diff                           0.0
mean_order_reviews_score_of_given_client    0.0
average_order_price_of_client               0.0
non_cancelled_orders_count                  0.0
total_order_price_of_client                 0.0
dtype: float64


A ce qu'on peut voir ci-dessus, il n'y a pas de valeurs manquantes dans le jeu de données.

### Supprimer les clients avec des commandes annulées et aucune commande non annulée

En effet, seuls les vrais clients nous intéressent ici.

In [8]:
import pandas as pd

# Supposons que 'data' est déjà chargé

# Calculer le nombre initial de lignes
initial_rows = data.shape[0]

# Supprimer les clients avec des commandes annulées et aucune commande non annulée
filtered_data = data[~((data['non_cancelled_orders_count'] == 0))]

# Calculer le nombre de lignes après le filtrage
final_rows = filtered_data.shape[0]

# Calculer le nombre de lignes supprimées
rows_removed = initial_rows - final_rows

# Afficher le nombre de lignes supprimées
print(f"Nombre initial de lignes: {initial_rows}")
print(f"Nombre final de lignes: {final_rows}")
print(f"Nombre de lignes supprimées: {rows_removed}")

# Continuer avec les données filtrées
data = filtered_data


Nombre initial de lignes: 94989
Nombre final de lignes: 94989
Nombre de lignes supprimées: 0


## Retirer les valeurs aberrantes

#### Retirer les valeurs négatives 

In [9]:
import pandas as pd

# Create a dictionary to hold the counts of negative values per column
negative_counts = {}

columns_to_check = [
    'non_cancelled_orders_count',
    'days_since_last_order',
    'average_days_diff',
    'std_dev_days_diff',
    'mean_order_reviews_score_of_given_client',
    'average_order_price_of_client',
    'total_order_price_of_client'
]

# Loop through each column to find the count of negative values
for column in columns_to_check:
    count = (data[column] < 0).sum()
    negative_counts[column] = count

# Print the count of negative values for each column
for column, count in negative_counts.items():
    print(f"Negative values in {column}: {count}")


Negative values in non_cancelled_orders_count: 0
Negative values in days_since_last_order: 0
Negative values in average_days_diff: 0
Negative values in std_dev_days_diff: 0
Negative values in mean_order_reviews_score_of_given_client: 0
Negative values in average_order_price_of_client: 0
Negative values in total_order_price_of_client: 0


#### Retirer les valeurs nulles ou négatives

Etant donné que nous avons désormais seulement des clients qui on au moins effectués une seule commande, les valeurs des colonnes average_order_price_of_client et total_price_of_non_cancelled_orders sont necessairement supérieures à zéro, alors si présentes nous supprimons la ligne correspondante :

In [10]:
import pandas as pd

# Assuming 'data' is a Pandas DataFrame already loaded with your dataset
# Replace 'your_data.csv' with the path to your CSV file if needed to load it
# data = pd.read_csv('your_data.csv')

# Initial count of rows in the original dataset
original_count = data.shape[0]

# Define the conditions to check
condition = (
    (data['average_order_price_of_client'] <= 0) |
    (data['total_order_price_of_client'] <= 0)
)

# Filter out rows that meet the condition
data_cleaned = data[~condition]

# New count of rows after cleaning
cleaned_count = data_cleaned.shape[0]

# Calculate the number of rows removed
rows_removed = original_count - cleaned_count

print("Nombre de lignes supprimées: ", rows_removed)

Nombre de lignes supprimées:  0


In [11]:
print(data_cleaned.shape)

(94989, 8)


Nous avons désormais un jeu de données traité et nettoyé que nous pouvons utiliser sereinement pour la segmentation.

In [ ]:
# Définir le chemin de sauvegarde
save_path = r"C:\Users\pdevi\OneDrive\Desktop\OpenClassrooms\Projet_3\DATA_LAST\data_cleaned_0.csv"

# Sauvegarder le DataFrame au format CSV
data_cleaned.to_csv(save_path, index=False)

print(f"DataFrame saved to {save_path}")